## Hello everybody 😃 😃 welcome back


In [ ]:
import os
import cv2 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Image Size
IMG_ROWS, IMG_COLS = 224, 224 # input image dimensions
NB_CLASSES =  10 # number of outputs = number of digits

## (01)   ACQUISITION DES DONNEES - IMAGES DES FEUILLES DES PLANTES. 

In [ ]:
########################################### function for plotting images

def plot_images(images, total_images=100, rows=20, cols=5, fsize=(20,100), titre='Image'):
    
    fig = plt.figure(figsize=fsize) # create a new figure window
    
    for i in range(total_images): # display images
        # subplot : 33 rows and 5 columns
        img_grid = fig.add_subplot(rows, cols, i+1)
        # plot features as image
        img_grid.imshow(images[i])
        
        plt.title(titre)


In [ ]:
############################################ function for resizng images
       
def preprocess_image(image, image_height=IMG_ROWS, image_width=IMG_COLS):

    return cv2.resize(image, (image_height, image_width))

############################################ function for reading images 
       
def read_images (path , sz= None ):
    
    print('\nCHARGEMENT DES IMAGES DE LA BASE .......................!\n') 

    X,y = [], []
    
    for dirname , dirnames , filenames in os.walk(path):
        
        c = 0
        
        for subdirname in dirnames :
            
            subject_path = os. path . join ( dirname , subdirname )
            
            for filename in os. listdir ( subject_path ):
                
                im = Image.open(os.path.join(subject_path, filename))
                #im = im.convert ("L")

                if (sz is not None ):
                    im = im.resize (sz , Image.ANTIALIAS ) 
                    
                im = np.array(im)
                im = preprocess_image(im, IMG_ROWS, IMG_COLS)
                X.append(im)
                y.append (c)  
                
            c = c+1
            
    return [X,y]


In [ ]:
############################################ Read And Resize test images

print('\n\nLECTURE DES IMAGES DE LA BASE D\'APPRENTISSAGE........!') 

[X_train, y_train] = read_images("./Data/Potato/TrainData") # Potato

print('\nAFFICHAGE DE QUELQUES IMAGES DE LA BASE.................!')
plot_images(X_train, 2, 1, 2,(10, 50), titre='Base D\'Apprentissage')
plt.show()
print('\nFIN D\'AFFICHAGE DES IMAGES DE LA BASE...................!\n')

In [ ]:
############################################################# Train Data

images_train = X_train
images_train = np.asarray(images_train)

# Train targets
train_features = images_train
train_targets = y_train 

from keras.utils import to_categorical
# convert class vectors to binary class matrices
train_targets = to_categorical(train_targets, NB_CLASSES)

print('\nNORMALISATION DES BASES DE TEST ET D\'APPRENTISSAGE.\n')

########################################################## Normalisation

train_features = train_features.astype('float32')

mean_vals = np.mean(train_features, axis=0)
std_val = np.std(train_features)
train_features = (train_features - mean_vals)/std_val

train_features = train_features.reshape(train_features.shape[0], IMG_ROWS, IMG_COLS, 3)
print("train_features.shape     >==============<> : {}".format(train_features.shape))
print("train_targets.shape      >==============<> : {}".format(train_targets.shape))

In [ ]:
########################################################################
#(02)        CLASSIFICATION : CREATION DU MODELE DE PREDICTION         #
########################################################################

In [ ]:
from keras import optimizers
from time import time
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.initializers import he_normal

In [ ]:
######################################################## Build The Model
class VGGNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        # Block N°1
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal(), input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # Block N°2
        model.add(Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(128, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # Block N°3
        model.add(Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(256, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # Block N°4
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # Block N°5
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_initializer=he_normal()))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # Fully connected layers
        model.add(Flatten())
        model.add(Dense(4096, activation="relu", kernel_initializer=he_normal()))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation="relu", kernel_initializer=he_normal()))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation="softmax"))
        
        return model



In [ ]:
########################################################## Use The Model

OPTIMIZER=optimizers.Adam()
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 3)
NB_EPOCH = 1
BATCH_SIZE = 64
VERBOSE = 1

# initialize the optimizer and model
model = VGGNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

t_start = time()
history = model.fit(train_features, train_targets, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE)
time_full_train = time() - t_start

print("\nTEMPS D\'APPRENTISSAGE DU CLASSIFIEUR >====<> : %0.2fs " % (time_full_train))
model.save("VGGNet_16_groupe_2.h5")
print('\nENREGISTRER LE MODELE .\n') 

# Measure of VGGNet16 for one dataset patato
1.  86.02%
2.  86.26%
3.  85.95%
4.  86.88%
5.  84.88%
6.  84.42%
7.  83.26%
8.  84.42%
9.  84.79%
10. 83.49%
Result of VGGNet16 for ten test = 85.25%